##### Note_SQL.ipynb

- 2020年9月14日：EXPORT時にスレッド数を指定してマルチコアで行うことで、所要時間を減らす。

In [ ]:
EXPORT <テーブル名1>, <テーブル名2>,... AS <拡張子> INTO <出力先のパス> THREAD <スレッド数>;

- 2020年9月18日：パーティションで分けてINSERTをマルチコアで行うことで、所要時間を減らす。

In [ ]:

ALTER TABLE


- 2020年9月24日：SAP HANAのデルタマージの遅延を解消するためのクエリ
- 参考：https://books.google.co.jp/books?id=BUY4DwAAQBAJ&pg=PA59&lpg=PA59&dq=hana+%E3%82%B9%E3%83%9E%E3%83%BC%E3%83%88%E3%83%9E%E3%83%BC%E3%82%B8+merge+delta+of&source=bl&ots=356sqJnFzC&sig=ACfU3U1tUXnbPwPrZfodkTuunfdahgaBWw&hl=ja&sa=X&ved=2ahUKEwjoo6eM-oTsAhXWxIsBHdVfBU8Q6AEwB3oECAEQAQ#v=onepage&q=hana%20%E3%82%B9%E3%83%9E%E3%83%BC%E3%83%88%E3%83%9E%E3%83%BC%E3%82%B8%20merge%20delta%20of&f=false

In [ ]:
/* クリティカルマージとは、デルタマージの大きさが特定の閾値を超えると、HANAが自動的に行うマージのこと。 */
/* スマートマージ */
MERGE DELTA OF <テーブル名> WITH PARAMETERS (‘SMART_MERGE’ = ‘ON');
/* ハードマージ */
MERGE DELTA OF <テーブル名> ;
/* フォースマージは以下のどちらか。 */
MERGE DELTA OF <テーブル名> FORCE REBUILD;
MERGE DELTA OF <テーブル名> WITH PARAMETERS (‘FORCED_MERGE’ = ‘ON');

- 2020年9月26日：ORDER BYの順序→前に書いた方を基準にソートされる。

In [ ]:
SELECT winner, subject FROM nobel WHERE yr=1984
 ORDER BY subject IN (
  'Physics', ‘Chemistry'
) ASC, subject ASC, winner ASC;

- 2020年9月26日：'Argentina'または'Australia'を含む大陸にある国の、国名と大陸を表示する。国名順に表示する。

In [ ]:
SELECT name, continent FROM world WHERE continent IN (
  SELECT continent FROM world WHERE name IN ('Argentina', 'Australia’)
)
ORDER BY name ASC;

- 2020年9月26日：人口がカナダ Canadaよりも多く、ポーランドPolandよりも少ない国の、国名と人口を表示する。

In [ ]:
SELECT name, population FROM world WHERE population BETWEEN (
  SELECT population FROM world WHERE name = ‘Poland’
) 
AND (
  SELECT population FROM world WHERE name = ‘Canada’
);

- 2020年9月26日：ヨーロッパの各国について 国名と人口を表示する。ドイツ人口の何％かで表示する。

In [ ]:
SELECT name, CONCAT(
  ROUND(
    population/(
      SELECT population FROM world WHERE name = ‘Germany'
    )*100
  ),’%’
) 
FROM world WHERE continent = 'Europe';

- 2020年9月26日：ヨーロッパのどの国のGDPよりも大きなGDPを持つ国の国名だけを表示する。（GDPがNULL の国も有る：「gdp > 0」で処理。）

In [ ]:
SELECT name FROM world
WHERE gdp > ALL(
  SELECT gdp FROM world
  WHERE gdp > 0 AND continent = 'Europe'
)

- 2020年9月26日：各大陸のもっとも大きな国（面積で）の大陸、国名、面積を表示する。
- （自己相関クエリー というテクニックを利用して解く。
- 相関サブクエリーの２重ループの様に機能する：内側のサブクエリーは一度に１行ずつ外側のクエリーの関連するレコードにアクセスする。内と外で同じ名前のテーブルを区別するテクニックとして、外側のクエリーのテーブルと内側のサブクエリーのテーブルの名前を付け替える。
- ）

In [ ]:
SELECT continent, name, area FROM world x
WHERE area >= ALL(
  SELECT area FROM world y
  WHERE y.continent=x.continent AND area > 0
)

- 2020年9月26日：各大陸の中でアルファベット順で先頭になる国の大陸と国名を表示する。

In [ ]:
SELECT continent, name FROM world x
WHERE name <= ALL(
  SELECT name FROM world y
  WHERE y.continent=x.continent
)

- 2020年9月26日：世界の総人口を表示。（各国の人口を合計）

In [ ]:
SELECT SUM(population) FROM world

- 2020年9月26日：各大陸continentごとに大陸名continentとそこの国の数を表示する。

In [ ]:
SELECT continent, COUNT(continent) FROM world GROUP BY continent

- 2020年9月26日：その大陸の各国の人口の合計が100000000人以上の大陸のリストを表示する

In [ ]:
SELECT t1.continent 
FROM (SELECT continent, SUM(population) AS S FROM world 
GROUP BY continent) AS t1 
WHERE t1.S >= '100000000';

- 2020年9月26日：Marioという名前の選手のゴールについて、team1, team2 , player を表示する。

In [ ]:
SELECT game.team1, game.team2, goal.player 
FROM game JOIN goal ON (
  game.id = goal.matchid
) 
WHERE goal.player LIKE 'Mario%'

##### 以下は、https://sqlzoo.net/wiki/More_JOIN_operations/jaを解いたもの。
- 2020年9月26日：映画「エイリアン」'Alien'の出演者リストを表示。（movie、casting、actor）

In [ ]:
SELECT actor.name 
FROM (
  SELECT * FROM movie 
  INNER JOIN casting ON (
    movie.id = casting.movieid
  ) 
  WHERE movie.title = ‘Alien’
) AS t1 INNER JOIN actor ON (
  t1.actorid = actor.id
);

- 2020年9月26日：'Harrison Ford'  ハリソン＝フォードが出演した映画のリストを表示する。（movie、casting、actor）

In [ ]:
/* 3. 映画のIDから映画のタイトルを取得。 */
SELECT movie.title 
FROM movie INNER JOIN (
  /* 2. 俳優のIDから映画のIDを取得。 */
  SELECT casting.movieid FROM casting 
  WHERE casting.actorid = (
    /* 1. まず、俳優の名前から俳優のIDを取得。 */
    SELECT actor.id FROM actor WHERE name = 'Harrison Ford'
  )
)AS t1 ON (
  movie.id = t1.movieid
);

- 2020年9月26日：'Art Garfunkel' アート＝ガーファンクルと一緒に仕事をした人々をすべて表示。（movie、casting、actor）

In [ ]:
/* 4. 俳優のIDから俳優の名前を取得。 */
SELECT actor.name 
FROM actor INNER JOIN (
  /* 3. 映画のIDから俳優のIDを取得。 */
  SELECT DISTINCT casting.actorid FROM casting 
  WHERE casting.movieid IN (
    /* 2. 俳優のIDから映画のIDを取得。 */
    SELECT movieid FROM casting 
    WHERE casting.actorid = (
      /* 1. まず、俳優の名前から俳優のIDを取得。 */
      SELECT actor.id FROM actor WHERE name = 'Art Garfunkel'
    )
  )
)AS t1 ON (
  actor.id = t1.actorid
)
/* 5. 特定の俳優を除外。 */
WHERE name <> 'Art Garfunkel'
;

- 2020年9月26日：'Art Garfunkel' アート＝ガーファンクルと一緒に仕事をした人々をすべて表示。　にて、監督も含めたもの。（movie、casting、actor）

In [ ]:
/* 4. 俳優のIDから俳優の名前を取得。 */
SELECT actor.name 
FROM actor INNER JOIN (
  /* 3. 映画のIDから俳優のIDを取得。 */
  SELECT DISTINCT casting.actorid FROM casting 
  WHERE casting.movieid IN (
    /* 2. 俳優のIDから映画のIDを取得。 */
    SELECT movieid FROM casting 
    WHERE casting.actorid = (
      /* 1. まず、俳優の名前から俳優のIDを取得。 */
      SELECT actor.id FROM actor WHERE name = 'Art Garfunkel'
    )
  )
)AS t1 ON (
  actor.id = t1.actorid
)
/* 5. 特定の俳優を除外。 */
WHERE name <> 'Art Garfunkel'
UNION 
/* 4. 映画の監督(ID)から映画の監督の名前を取得。 */
SELECT actor.name
FROM actor INNER JOIN (
  /* 3. 映画のIDから映画の監督(ID)を取得。 */
  SELECT movie.director 
  FROM movie INNER JOIN (
    /* 2. 俳優のIDから映画のIDを取得。 */
    SELECT casting.movieid FROM casting 
    WHERE casting.actorid = (
      /* 1. まず、俳優の名前から俳優のIDを取得。 */
      SELECT actor.id FROM actor WHERE name = 'Art Garfunkel'
    )
  ) AS t2 ON (
    movie.id = t2.movieid
  )
) AS t3 ON (
  actor.id = t3.director
)
;

##### SQLite
- SQLiteはローカルのファイルを直接データベースとして扱うので、複数のサーバーとネットワークごしに接続するのには不向きです。Webサーバーを複数台起動する場合などはMySQLやPostgreSQLなどを使うほうが良いでしょう。